In [ ]:
from ayx import Alteryx
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from pptx import Presentation
from pptx.chart.data import ChartData
from pptx.chart.datalabel import DataLabel
from pptx.enum.chart import XL_CHART_TYPE
from pptx.util import Inches
import datetime
from pptx.dml.color import RGBColor
from pptx.enum.chart import XL_LABEL_POSITION
from pptx.util import Pt
from pptx.enum.chart import XL_TICK_MARK
from pptx.enum.text import MSO_ANCHOR, MSO_AUTO_SIZE,PP_ALIGN
from pptx.enum.shapes import MSO_SHAPE
from pptx.enum.chart import XL_LEGEND_POSITION
import time

In [ ]:
gsdb_plant_acronym_dict = {'0117A': 'Dagenham Engine Plant', '0115A': 'Bridgend Engine Plant', 
                           'FI05D': 'Taubate Engine Plant', '0146A': 'Valencia Engine Plant', 
                           '0100A': 'Cologne Engine Plant', 'FWSTA': 'Craiova Engine Plant', 
                           'FKARE': 'Sanand Engine Plant', '1055R': 'Chennai Engine Plant', 
                           '4125A': 'Irapuato Transmission Plant', '6653A': 'Bordeaux Transmission Plant', 
                           'AX9WA': 'Strundale Engine Plant', 
                           'EF06A': 'Lima Engine Plant', 'EF0AA': 'Romeo Engine Plant',
                           'EF15A': 'Windsor Engine Plant', 'EF17A': 'Essex Engine Plant', 
                           'TC02A': 'Sharonville Transmission Plant', 'TC04A': 'Livonia Transmission Plant', 
                           'TC11A': 'Vandyke Transmission Plant',
                           'EF18A': 'Chihuahua Engine Plant',
                           'EE01A':'Rawsonville Plant' , 'EF01A': 'Cleveland Engine Plant', 
                           'EF03A': 'Dearborn Engine Plant'}

In [ ]:
purch_data = pd.DataFrame(Alteryx.read("#1"))
gtsv_data = pd.DataFrame(Alteryx.read("#2"))
print ("# of Rows in Purchase Data", len(purch_data))
gtsv_data = gtsv_data.loc[gtsv_data['Tool_Type']== 'Perishable'].copy().reset_index()
gtsv_data['start_date'] = pd.to_datetime(gtsv_data['start_date']).dt.date
gtsv_data['end_date'] = pd.to_datetime(gtsv_data['end_date']).dt.date
purch_data['start_date'] = pd.to_datetime(purch_data['start_date']).dt.date
purch_data['end_date'] = pd.to_datetime(purch_data['end_date']).dt.date

start_date = str(gtsv_data['start_date'][0])
end_date = str(gtsv_data['end_date'][0])


gtsv_data = gtsv_data.rename(columns={"DESCRIPTION": "MACHINING_FEATURE"})
gtsv_data['T_Number'] = gtsv_data['T_Number'].fillna('Undefined')

purch_data = purch_data.rename(columns = {'DEPARTMENT': 'AREA', 'ITEM_ID': 'CHILD_ID'})
#purch_data = purch_data[purch_data['DURABLE_OR_PERISHABLE'] == 'Perishable']
purch_data['EXTENDED_COST'] = -1*purch_data['EXTENDED_COST']

ref_t_number = gtsv_data.groupby(['GSDB', 'CHILD_ID']).agg({'TOOLNAME': lambda x: ', '.join(x.unique()),'T_Number': lambda y: ', '.join(y.unique())}).reset_index()

purch_data = purch_data.reset_index()
print ("# of Rows in Purchase Data", len(purch_data))
purch_data = pd.merge(purch_data,ref_t_number, on = ['GSDB', 'CHILD_ID'], how = 'left')
print ("# of Rows in Purchase Data", len(purch_data))
print ("Success")

In [ ]:
def first_slide (filtered_data, area ,prs,plant,report_type = 'YTD' ):
    #title_slide_layout = prs.slide_layouts[1]
    slide = prs.slides.add_slide(prs.slide_layouts[6])
    
    title_left, title_top, title_width, title_height = Inches(2), Inches(0.5), Inches(9.667), Inches(1)
    title_shape = slide.shapes.add_textbox(title_left, title_top, title_width, title_height)
    title_text_frame = title_shape.text_frame
    title = title_text_frame.paragraphs[0]
    title.font.size = Pt(32)
    title.alignment = PP_ALIGN.LEFT
    title.font.name = 'Times New Roman'
    title.text  =  plant 
    
    title_left, title_top, title_width, title_height = Inches(2), Inches(2), Inches(9.667), Inches(1)
    title_shape = slide.shapes.add_textbox(title_left, title_top, title_width, title_height)
    title_text_frame = title_shape.text_frame
    title = title_text_frame.paragraphs[0]
    title.font.size = Pt(22)
    title.font.bold = True
    title.font.italics = True
    title.font.underline = True
    title.alignment = PP_ALIGN.CENTER
    title.font.name = 'Times New Roman'
    title.text  =  str(area) + " Tooling Performance Report "  + " for (" + start_date + " - " + end_date + ")"
    
    title_left, title_top, title_width, title_height = Inches(2), Inches(3), Inches(9.667), Inches(1)
    title_shape = slide.shapes.add_textbox(title_left, title_top, title_width, title_height)
    title_text_frame = title_shape.text_frame
    title = title_text_frame.paragraphs[0]
    title.font.size = Pt(14)
    title.alignment = PP_ALIGN.CENTER
    title.font.name = 'Times New Roman'
    now = datetime.date.today().strftime('%B %d, %Y')
    title.text  =  'Report Generated on: ' + str(now)
    
    if not filtered_data.empty:
        return slide
    else :
        return 0
    
   

In [ ]:
def top_purch(filtered_data, prs, prs_name,report_type = 'YTD', plant = 'DEP'):
    title_slide_layout = prs.slide_layouts[6]
    slide = prs.slides.add_slide(title_slide_layout)
    title_left, title_top, title_width, title_height = Inches(2), Inches(0.5), Inches(9.667), Inches(1)
    title_shape = slide.shapes.add_textbox(title_left, title_top, title_width, title_height)
    title_text_frame = title_shape.text_frame
    title = title_text_frame.paragraphs[0]
    title.font.size = Pt(22)
    title.alignment = PP_ALIGN.LEFT
    title.font.name = 'Calibri Light'
    title.text = prs_name 
    filtered_data = filtered_data.fillna('NotAvailable')
    filtered_data = filtered_data.groupby(['NAME']).agg({'EXTENDED_COST':'sum', 'QUANTITY':'sum', 
                                                                     'TOOLNAME': lambda col: ' , '.join(col.unique()),
                                                                     'T_Number': lambda col: ' , '.join(col.unique()),
                                                                     'PURCHASE_TYPE': lambda col: ' , '.join(col.unique())}).reset_index()
    filtered_data = filtered_data.sort_values(by = ['EXTENDED_COST'],ascending = False)
    filtered_data['EXTENDED_COST'] = filtered_data['EXTENDED_COST'].round().astype(int)
    
    filtered_data = filtered_data.head(5)
    
    if not filtered_data.empty:
        # define chart data ---------------------
        chart_data = ChartData()
        tool_categories = filtered_data['NAME']#.unique()
        extended_cost = filtered_data['EXTENDED_COST']

        chart_data.categories = tool_categories#[replacement['TOOLNAME'][0], replacement['TOOLNAME'][1], replacement['TOOLNAME'][2], replacement['TOOLNAME'][3], replacement['TOOLNAME'][4]]
        labels = chart_data.categories
        chart_data.add_series('Replacement Cost', extended_cost)

        # add chart to slide --------------------
        left, top,width, height = Inches(1.665), Inches(1.5), Inches(10), Inches(2)
        chart = slide.shapes.add_chart(XL_CHART_TYPE.COLUMN_CLUSTERED, left, top,width, height, chart_data).chart
        chart.has_title = True
        chart.chart_title.has_text_frame=True
        chart.chart_title.text_frame.text=prs_name
        chart.chart_title.text_frame.paragraphs[0].font.size = Pt(10)
        chart.value_axis.has_minor_gridlines = False
        chart.value_axis.has_major_gridlines = False
        # add data labels to chart --------------------
        value_label = chart.plots[0]
        value_label.gap_width = 25
        bars = value_label.series[0]
        fill = bars.format.fill
        fill.solid()
        fill.fore_color.rgb = RGBColor(20, 61, 141)
        value_label.has_data_labels = True
        data_labels = value_label.data_labels
        data_labels.font.color.rgb = RGBColor(0, 0, 0)
        data_labels.font.size = Pt (10)
        data_labels.position = XL_LABEL_POSITION.OUTSIDE_END
        # x-axis tick labels
        category_axis = chart.category_axis
        category_axis.has_major_gridlines = False
        category_axis.tick_labels.font.size = Pt(10)
        # y-axis tick labels
        value_axis = chart.value_axis
        tick_labels = value_axis.tick_labels
        tick_labels.font.size = Pt(10)

        row_num = 1+len(filtered_data)
        col_num = 7

        left = Inches(1)
        top = Inches(3.5)
        width = Inches(14)
        height = Inches(3.2)
        # add table to slide --------------------
        table = slide.shapes.add_table(row_num, col_num, left, top, width, height).table
        # set column widths
        table.columns[0].width = Inches(1.35)
        table.columns[1].width = Inches(0.75)
        table.columns[2].width = Inches(0.75)
        table.columns[3].width = Inches(1)
        table.columns[4].width = Inches(2.5)
        table.columns[5].width = Inches(1.5)
        table.columns[6].width = Inches(3.5)
        # top row height
        table.rows[0].height = Inches(0.25)

        for row in range(row_num):
            for col in range(col_num):
                text_frame = table.cell(row,col).text_frame
                text_frame.clear()
                p = text_frame.paragraphs[0]
                if row == 0:
                    p.font.size = Pt(10)
                    p.alignment = PP_ALIGN.CENTER
                else:
                    p.font.size = Pt(10)
                    p.alignment = PP_ALIGN.CENTER
        # write body cells
                if row >0:
                    if col == 0:
                        table.cell(row, col).text = str(filtered_data['NAME'].iloc[row-1])
                    elif col == 1:
                        table.cell(row, col).text = str(filtered_data['EXTENDED_COST'].iloc[row-1])
                    elif col == 2:
                        table.cell(row, col).text = str(filtered_data['QUANTITY'].iloc[row-1])
                    elif col == 3:
                        table.cell(row, col).text = str(filtered_data['PURCHASE_TYPE'].iloc[row-1])
                    elif col == 4:
                        table.cell(row, col).text = str(filtered_data['TOOLNAME'].iloc[row-1])
                    elif col == 5:
                        table.cell(row, col).text = str(filtered_data['T_Number'].iloc[row-1])
            # write column headings
            table.cell(0, 0).text = 'NAME'
            table.cell(0, 1).text = 'Total Purchase Cost (in $)'
            table.cell(0, 2).text = 'Quantity Purchased'
            table.cell(0, 3).text = 'Purchase Type'
            table.cell(0, 4).text = 'Used in Tool'
            table.cell(0, 5).text = 'Ref T Number'
            table.cell(0, 6).text = 'Usage Type'
        
        def iter_cells(table):
            for row in table.rows:
                for cell in row.cells:
                    yield cell

        for cell in iter_cells(table):
            for paragraph in cell.text_frame.paragraphs:
                for run in paragraph.runs:
                    run.font.size = Pt(10)
                    run.font.name = 'Times New Roman'
        return slide
    else:
        return 0
    

In [ ]:
def top_n_issues(filtered_data, prs, prs_name,report_type = 'YTD', plant = 'DEP'):
    title_slide_layout = prs.slide_layouts[6]
    slide = prs.slides.add_slide(title_slide_layout)
    title_left, title_top, title_width, title_height = Inches(2), Inches(0.5), Inches(9.667), Inches(1)
    title_shape = slide.shapes.add_textbox(title_left, title_top, title_width, title_height)
    title_text_frame = title_shape.text_frame
    title = title_text_frame.paragraphs[0]
    title.font.size = Pt(22)
    title.alignment = PP_ALIGN.LEFT
    title.font.name = 'Calibri Light'
    title.text = prs_name
    
    filtered_data = filtered_data.groupby(['TOOLNAME','AREA','MACHINING_FEATURE', 'T_Number']).agg({'CHG-LOSS-USD':sum, 'CHNG_ID': 'nunique','LAST_UPDATED_TCF':'mean', 'END_COUNT': 'mean'}).reset_index()
    filtered_data = filtered_data.rename(columns = {'CHNG_ID': 'NUM_OF_OCCURENCES'} )
    
    filtered_data['CHG-LOSS-USD'] = filtered_data['CHG-LOSS-USD'].round().astype(int)
    filtered_data['END_COUNT'] = filtered_data['END_COUNT']/filtered_data['LAST_UPDATED_TCF']*100
    filtered_data = filtered_data.replace([np.inf, -np.inf], np.nan)
    filtered_data['END_COUNT'] = filtered_data['END_COUNT'].fillna(0)
    filtered_data['END_COUNT'] = filtered_data['END_COUNT'].round().astype(int)
    filtered_data = filtered_data.sort_values(by = 'CHG-LOSS-USD', ascending = False)
    
    filtered_data = filtered_data.head(5)
    if not filtered_data.empty:
        # define chart data ---------------------
        chart_data = ChartData()
        tool_categories = filtered_data['TOOLNAME']#.unique()
        replacement_cost = filtered_data['CHG-LOSS-USD']

        chart_data.categories = tool_categories#[replacement['TOOLNAME'][0], replacement['TOOLNAME'][1], replacement['TOOLNAME'][2], replacement['TOOLNAME'][3], replacement['TOOLNAME'][4]]
        labels = chart_data.categories
        chart_data.add_series('Replacement Cost', replacement_cost)

        # add chart to slide --------------------
        left, top,width, height = Inches(1.665), Inches(1.5), Inches(10), Inches(2)
        chart = slide.shapes.add_chart(XL_CHART_TYPE.COLUMN_CLUSTERED, left, top,width, height, chart_data).chart
        chart.has_title = True
        chart.chart_title.has_text_frame=True
        chart.chart_title.text_frame.text=prs_name
        chart.chart_title.text_frame.paragraphs[0].font.size = Pt(10)
        chart.value_axis.has_minor_gridlines = False
        chart.value_axis.has_major_gridlines = False
        # add data labels to chart --------------------
        value_label = chart.plots[0]
        value_label.gap_width = 25
        bars = value_label.series[0]
        fill = bars.format.fill
        fill.solid()
        fill.fore_color.rgb = RGBColor(20, 61, 141)
        value_label.has_data_labels = True
        data_labels = value_label.data_labels
        data_labels.font.color.rgb = RGBColor(0, 0, 0)
        data_labels.font.size = Pt (10)
        data_labels.position = XL_LABEL_POSITION.OUTSIDE_END
        # x-axis tick labels
        category_axis = chart.category_axis
        category_axis.has_major_gridlines = False
        category_axis.tick_labels.font.size = Pt(10)
        # y-axis tick labels
        value_axis = chart.value_axis
        tick_labels = value_axis.tick_labels
        tick_labels.font.size = Pt(10)

        row_num = 1+len(filtered_data)
        col_num = 8

        left = Inches(0.5)
        top = Inches(4)
        width = Inches(14)
        height = Inches(3.2)
        # add table to slide --------------------
        table = slide.shapes.add_table(row_num, col_num, left, top, width, height).table
        # set column widths
        table.columns[0].width = Inches(1.35)
        table.columns[1].width = Inches(1)
        table.columns[2].width = Inches(1)
        table.columns[3].width = Inches(1)
        table.columns[4].width = Inches(1)
        table.columns[5].width = Inches(2)
        table.columns[6].width = Inches(2.5)
        table.columns[7].width = Inches(2.5)
        # top row height
        table.rows[0].height = Inches(0.25)

        for row in range(row_num):
            for col in range(col_num):
                text_frame = table.cell(row,col).text_frame
                text_frame.clear()
                p = text_frame.paragraphs[0]
                p.font.size = Pt(10)
                p.alignment = PP_ALIGN.CENTER
                if row>0:
                    if col == 0:
                        table.cell(row, col).text = str(filtered_data['TOOLNAME'].iloc[row-1])
                    elif col == 1:
                        table.cell(row, col).text = str(filtered_data['CHG-LOSS-USD'].iloc[row-1])
                    elif col == 2:
                        table.cell(row, col).text = str(filtered_data['NUM_OF_OCCURENCES'].iloc[row-1])
                    elif col == 3:
                        table.cell(row, col).text = str(filtered_data['T_Number'].iloc[row-1])
                    elif col == 4:
                        table.cell(row, col).text = str(filtered_data['END_COUNT'].iloc[row-1])
                    elif col == 5:
                        table.cell(row, col).text = str(filtered_data['MACHINING_FEATURE'].iloc[row-1])
            # write column headings
            table.cell(0, 0).text = 'Assembly'
            table.cell(0, 1).text = 'Total Cost (in $)'
            table.cell(0, 2).text = 'Num of Occurences'
            table.cell(0, 3).text = 'T Number'
            table.cell(0, 4).text = '% Tool Life Achieved'
            table.cell(0, 5).text = 'Machining Feature'
            table.cell(0, 6).text = 'Work Stream'
            table.cell(0, 7).text = 'Action Taken'

        def iter_cells(table):
            for row in table.rows:
                for cell in row.cells:
                    yield cell

        for cell in iter_cells(table):
            for paragraph in cell.text_frame.paragraphs:
                for run in paragraph.runs:
                    run.font.size = Pt(8)
                    run.font.name = 'Times New Roman'
                    
        return slide
    else :
        return 0

    

In [ ]:
import re
for gsdb in gtsv_data['GSDB'].unique():
    #print ("GSDB", gsdb)
    plant_data = gtsv_data[gtsv_data['GSDB'] == gsdb]
    plant_data['AREA'] = plant_data['AREA'].map(lambda x: re.sub(r'\W+', '', x))
    
    for AREA in plant_data['AREA'].unique():
        prs = Presentation('//ecc9000106/qlik_prod8422/Descriptive_Analytics/old_drive/Analytics/Collaboration/BottleneckDetection/Tooling Project/Automated Power Point//Powerpoint_Layout.pptx')
        plant = gsdb_plant_acronym_dict[gsdb]
        area_data = plant_data[plant_data['AREA'] == AREA]
        area_purch = purch_data[purch_data['AREA'] == AREA]
        purch_name = AREA + " Top Spending for (" + str(start_date) + " - " + str(end_date) + " )" 
        #f = open("C:\Cutting Tools Dashboard\Combined Output\Powerpoint_Layout.pptx")
        content = first_slide(area_data,AREA,prs,plant)
        if len(area_purch)>1:
            content = top_purch(area_purch,prs,purch_name)
        
        for issue in area_data['CHG-TYPE'].unique():
            if issue !='Scheduled':
                tool_change_data = area_data.loc[area_data['CHG-TYPE']== issue].copy()
                if len(tool_change_data)>1:
                    plant = gsdb_plant_acronym_dict[gsdb]
                    name = ''
                    if issue!= 'Unscheduled':
                        name = ' Top ' + issue + " Assembly by Replacement Cost for (" + str(start_date) + " - " + str(end_date) + " )" 
                    else : 
                        name = ' Top ' + issue + " Assembly by LifeLost Cost for (" + str(start_date) + " - " + str(end_date) + " )" 
                    content = top_n_issues(tool_change_data,prs,name)
                    for LINE in tool_change_data['LINE'].unique():
                        line_data = tool_change_data.loc[tool_change_data['LINE']  == LINE].copy()
                        new_name = LINE + " "+name
                        content = top_n_issues(line_data,prs,new_name)
        #os.chdir(os.path.join('C:/Cutting Tools Dashboard/Combined Output'))
        name = '//ecc9000106/qlik_prod8422/Descriptive_Analytics/old_drive/Analytics/Collaboration/BottleneckDetection/Tooling Project/Automated Power Point/'+plant +  '_'+ AREA + '_'+ str(start_date)+ '_'+ str(end_date)+ ".pptx" 
        prs.save(name)
    #print ("Success")

In [ ]:
output_file_df = pd.DataFrame(columns = ['Name'])
output_file_df = output_file_df.append({'Name':name}, ignore_index = True)
Alteryx.write(output_file_df,1)